<a href="https://colab.research.google.com/github/abishekraja018/SDC-GENAI/blob/main/youtube_video_summariser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ✅ Step 1: Install Required Packages
!pip install youtube-transcript-api faiss-cpu sentence-transformers transformers -q

# ✅ Step 2: Imports
from youtube_transcript_api import YouTubeTranscriptApi
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from transformers import pipeline

# ✅ Step 3: Get Transcript from YouTube
def get_transcript(video_url):
    try:
        video_id = video_url.split("v=")[-1].split("&")[0]
        transcript_list = YouTubeTranscriptApi.get_transcript(video_id)
        full_text = " ".join([entry['text'] for entry in transcript_list])
        return full_text
    except Exception as e:
        return f"Error: {e}"

# ✅ Step 4: Chunk Text
def chunk_text(text, max_tokens=100):
    words = text.split()
    chunks = []
    for i in range(0, len(words), max_tokens):
        chunks.append(" ".join(words[i:i+max_tokens]))
    return chunks

# ✅ Step 5: Embed and Store in FAISS
def embed_chunks(chunks):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode(chunks)
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(np.array(embeddings))
    return index, chunks, model

# ✅ Step 6: RAG-style Retrieval + Summarization
def summarize_video(text):
    print("⏳ Chunking...")
    chunks = chunk_text(text)
    print(f"📚 Total Chunks: {len(chunks)}")

    print("📡 Embedding and Indexing...")
    index, all_chunks, embedder = embed_chunks(chunks)

    print("🔍 Retrieving top relevant chunks...")
    query = "Summarize this video"
    query_vec = embedder.encode([query])
    D, I = index.search(np.array(query_vec), k=5)
    top_chunks = [all_chunks[i] for i in I[0]]

    print("🧠 Generating summary with BART model...")
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    combined_text = " ".join(top_chunks)
    summary = summarizer(combined_text, max_length=150, min_length=40, do_sample=False)[0]['summary_text']
    return summary

# ✅ Step 7: Run the Pipeline
video_url = "https://www.youtube.com/watch?v=dQw4w9WgXcQ"  # 🔁 Replace with your video link
transcript = get_transcript(video_url)

if "Error" not in transcript:
    summary = summarize_video(transcript)
    print("\n✅ FINAL SUMMARY:\n")
    print(summary)
else:
    print(transcript)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.9 MB/s eta 0:00:00
⏳ Chunking...
📚 Total Chunks: 4
📡 Embedding and Indexing...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu



✅ FINAL SUMMARY:

"We've known each other for so long your heart's been aching but you're too sh to say inside we both know what's been going on we the game and we're going to play it" "I just want to tell you how I'm feeling got to make you understand" "We're no strangers to love you know the rules and so do I I full commitments while I'm thinking of you wouldn't get this from any other guy"


In [4]:
# ✅ Step 1: Install all required libraries
!pip install langchain langchain-community youtube-transcript-api faiss-cpu sentence-transformers transformers -q

# ✅ Step 2: Import everything
from youtube_transcript_api import YouTubeTranscriptApi
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

# ✅ Step 3: Define function to get YouTube transcript
def get_youtube_transcript(video_url):
    try:
        video_id = video_url.split("v=")[-1].split("&")[0]
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        return " ".join([entry['text'] for entry in transcript])
    except Exception as e:
        return f"Error: {e}"

# ✅ Step 4: Load local embeddings and LLM
def load_embeddings():
    return HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

def load_local_llm():
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn", tokenizer="facebook/bart-large-cnn")
    return HuggingFacePipeline(pipeline=summarizer)

# ✅ Step 5: Complete RAG pipeline
def rag_summarize_youtube(video_url):
    print("🔍 Extracting transcript...")
    transcript = get_youtube_transcript(video_url)
    if transcript.startswith("Error"):
        return transcript

    print("✂️ Splitting transcript into chunks...")
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = splitter.split_text(transcript)

    print("📦 Creating embeddings and FAISS vector store...")
    embeddings = load_embeddings()
    vectorstore = FAISS.from_texts(chunks, embedding=embeddings)

    print("🧠 Initializing RetrievalQA...")
    retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
    llm = load_local_llm()
    qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

    print("📝 Generating Summary...")
    summary = qa_chain.run("Summarize this video in detail.")
    return summary

# ✅ Step 6: Provide a YouTube video URL
video_url = "https://www.youtube.com/watch?v=dQw4w9WgXcQ"  # Replace with your own link
summary = rag_summarize_youtube(video_url)

# ✅ Step 7: Display the final summary
print("\n📌 Final Summary:\n")
print(summary)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00
🔍 Extracting transcript...
✂️ Splitting transcript into chunks...
📦 Creating embeddings and FAISS vector store...


<ipython-input-4-3c19a32c17d4>:24: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  return HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


🧠 Initializing RetrievalQA...


Device set to use cpu
<ipython-input-4-3c19a32c17d4>:28: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  return HuggingFacePipeline(pipeline=summarizer)
<ipython-input-4-3c19a32c17d4>:51: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  summary = qa_chain.run("Summarize this video in detail.")


📝 Generating Summary...

📌 Final Summary:

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don'st know, don't try to make up an answer. Never going to make you cry never going to sing goodbye going to tell a lie and hurt you.
